In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = input('Input utilize gpu-id (cpu:-1): ')
import sys
sys.path.append('utils/')

In [ ]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_integer('seq_length', 10,
                            """seq_length""")
tf.app.flags.DEFINE_integer('seq_start', 4,
                            """ start of seq generation""")
tf.app.flags.DEFINE_integer('batch_size', 1,
                            """batch size for training""")
tf.app.flags.DEFINE_integer('height', 128,
                            """height of input data""")
tf.app.flags.DEFINE_integer('width', 128,
                            """width of input data""")
tf.app.flags.DEFINE_string('warping', 'bilinear',
                            """method of warping""")
tf.app.flags.DEFINE_integer('gating_num', 3,
                            """# of experts""")
tf.app.flags.DEFINE_string('activation', 'softmax',
                            """gating activation""")
tf.app.flags.DEFINE_boolean('restore_all', True, 
                            """whether restore all or not """)
tf.app.flags.DEFINE_boolean('restore_gating', True, 
                            """whether restore gating or not """)
tf.app.flags.DEFINE_integer('h_conv_ksize', 3,
                            """kernel size of H""")
tf.app.flags.DEFINE_boolean('trainable_last', False, 
                            """train backtopixel conv(H) of MIM or not""")
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [ ]:
import layer_def as ld
from w_initializer import w_initializer
from Expert import MIM
from Gating import network_gate
network_gate = tf.make_template('gate', network_gate)

In [ ]:
def test(model1, model2, model3, model4, model5, model6, test_data):
    """
    model1: model path for translation
    model2: model path for rotation
    model3: model path for growth/decay
    model4: model path for H (decoder)
    model5: model path for gating
    model6: model path for the whole STMoE model
    test_data: np.array for test
    """
    x= tf.placeholder(tf.float32, [FLAGS.batch_size, FLAGS.height, FLAGS.width, FLAGS.seq_length])
    x_g = []
    weights, weights_last = [], []
    h_s, h_r, h_g, h_g_all= [], [], [], []

    hidden_state, hidden_state_diff, cell_state, cell_state_diff, st_memory = [], [], [], [], []
    hidden_state_r, hidden_state_diff_r, cell_state_r, cell_state_diff_r, st_memory_r = [], [], [], [], []
    hidden_state_s, hidden_state_diff_s, cell_state_s, cell_state_diff_s, st_memory_s = [], [], [], [], []  
    
    for i in range(FLAGS.seq_start - 1):
            
            with tf.variable_scope('expert2'):
                inputs = x[:, :, :, i]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_r, hidden_state_r, hidden_state_diff_r, cell_state_r, cell_state_diff_r, st_memory_r =                     MIM(inputs, i, hidden_state_r, hidden_state_diff_r, cell_state_r, cell_state_diff_r, st_memory_r, 3, [8, 8, 8], (3, 3), last_filter_size = FLAGS.h_conv_ksize, stride=1, tln = True)

            with tf.variable_scope('expert1'):
                inputs = x[:, :, :, i]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_s, hidden_state_s, hidden_state_diff_s, cell_state_s, cell_state_diff_s, st_memory_s =                     MIM(inputs, i, hidden_state_s, hidden_state_diff_s, cell_state_s, cell_state_diff_s, st_memory_s, 3, [8, 8, 8], (3, 3), last_filter_size = FLAGS.h_conv_ksize, stride=1, tln = True)
            
            with tf.variable_scope('expert3'):
                inputs = x[:, :, :, i]
                inputs = inputs[:, :, :, tf.newaxis]
                x_generate_g, hidden_state, hidden_state_diff, cell_state, cell_state_diff, st_memory =                     MIM(inputs, i, hidden_state, hidden_state_diff, cell_state, cell_state_diff, st_memory, 3, [8, 8, 8], (3, 3), last_filter_size = FLAGS.h_conv_ksize, stride=1, tln = True)
                
    for i in range(FLAGS.seq_length - FLAGS.seq_start):
        print('frame_{}'.format(i))
        if i == 0:
            inputs = x[:, :, :,  FLAGS.seq_start - 1]
            inputs = inputs[:, :, :, tf.newaxis]
            last_x = x[:, :, :, FLAGS.seq_start - 1]
            last_x = last_x[:, :, :, tf.newaxis]
            x_input = x[:, :, :, i:FLAGS.seq_start]
        else:
            inputs = x_g[-1]
            last_x = x_g[-1]
            x_gen = tf.stack(x_g)
            print(x_gen.shape)
            x_gen = tf.transpose(x_gen[:, :, :, :, 0], [1, 2, 3, 0])

            if i < FLAGS.seq_start:
                x_input = tf.concat([x[:, :, :, i:FLAGS.seq_start], x_gen[:,:, :, :i]], axis = 3)
            else:
                x_input = x_gen[:, :, :, i - FLAGS.seq_start:i]
        with tf.variable_scope('expert3'):
            x_generate_g, hidden_state, hidden_state_difff, cell_state, cell_state_diff, st_memory =                 MIM(inputs, i + FLAGS.seq_start - 1, hidden_state, hidden_state_diff, cell_state, cell_state_diff, st_memory, 3, [8, 8, 8], (3, 3), last_filter_size = FLAGS.h_conv_ksize, stride=1, tln = True)
            h_g.append(hidden_state[-1])
            
        with tf.variable_scope('expert1'):  
            x_generate_s, hidden_state_s, hidden_state_difff_s, cell_state_s, cell_state_diff_s, st_memory_s =                 MIM(inputs, i + FLAGS.seq_start - 1, hidden_state_s, hidden_state_diff_s, cell_state_s, cell_state_diff_s, st_memory_s, 3, [8, 8, 8], (3, 3), last_filter_size = FLAGS.h_conv_ksize, stride=1, tln = True)
            h_s.append(hidden_state_s[-1])
            
        with tf.variable_scope('expert2'):   
            x_generate_r, hidden_state_r, hidden_state_difff_r, cell_state_r, cell_state_diff_r, st_memory_r =                 MIM(inputs, i + FLAGS.seq_start - 1, hidden_state_r, hidden_state_diff_r, cell_state_r, cell_state_diff_r, st_memory_r, 3, [8, 8, 8], (3, 3), last_filter_size = FLAGS.h_conv_ksize, stride=1, tln = True)
            h_r.append(hidden_state_r[-1]) 

        
        with tf.variable_scope('gating_network', reuse = tf.AUTO_REUSE):
            if i == 0:
                x_input = x[:, :, :, i:i+FLAGS.seq_start]
            elif i < FLAGS.seq_start:
                x_input = tf.concat([x[:, :, :, i:FLAGS.seq_start], x_gen[:,:, :, :i]], axis = 3)
            else:
                x_input = x_gen[:, :, :, i - FLAGS.seq_start:i]
            
            weight = network_gate(x_input, FLAGS.gating_num, FLAGS.activation)
            h_generate = weight[:, :, :, 0][:, :, :, tf.newaxis] * h_s[-1] + weight[:, :, :, 1][:, :, :, tf.newaxis] * h_r[-1] + weight[:, :, :, 2][:, :, :, tf.newaxis] * h_g[-1] 
            
            x_generate = tf.layers.conv2d(h_generate,
                                     filters= 1,
                                     kernel_size=FLAGS.h_conv_ksize,
                                     strides=1,
                                     padding='same',
                                     kernel_initializer=w_initializer(8, 1),
                                     trainable = False,
                                     name="back_to_pixel")
            h_g_all.append(h_generate)
            x_g.append(x_generate)
            weights.append(weight)
            x_gen = tf.stack(x_g)
            x_gen = tf.transpose(x_gen[:, :, :, :, 0], [1, 2, 3, 0])
            
            w_generate_s = tf.layers.conv2d(tf.tile(weight[:, :, :, 0:1], [1, 1, 1, 8]),
                                 filters= 1,
                                 kernel_size=FLAGS.h_conv_ksize,
                                 strides=1,
                                 padding='same',
                                 kernel_initializer=w_initializer(8, 1),
                                 trainable = False,
                                 name="back_to_pixel")
            w_generate_r = tf.layers.conv2d(tf.tile(weight[:, :, :, 1:2], [1, 1, 1, 8]),
                                 filters= 1,
                                 kernel_size=FLAGS.h_conv_ksize,
                                 strides=1,
                                 padding='same',
                                 kernel_initializer=w_initializer(8, 1),
                                 trainable = False,
                                 name="back_to_pixel")
            w_generate_g = tf.layers.conv2d(tf.tile(weight[:, :, :, 2:3], [1, 1, 1, 8]),
                                 filters= 1,
                                 kernel_size=FLAGS.h_conv_ksize,
                                 strides=1,
                                 padding='same',
                                 kernel_initializer=w_initializer(8, 1),
                                 trainable = False,
                                 name="back_to_pixel")
                   
            w_generate = tf.concat([w_generate_s, w_generate_r, w_generate_g], axis = -1)
            
            weights_last.append(w_generate)


    x_g = tf.stack(x_g)
    x_g = tf.transpose(x_g[:, :, :, :, 0], [1, 2, 3, 0])
    print(x_g.shape)
    
    weights = tf.stack(weights)


    MSE = tf.losses.mean_squared_error(x[:, :, :, FLAGS.seq_start:], x_g[:, :, :, :])

    # SSIM
    SSIM = tf.image.ssim(x[:, :, :, FLAGS.seq_start:], x_g[:, :, :, :], 1)

    # List of all varables
    variables = tf.global_variables()
    # strat rinning operations on Graph
    sess = tf.Session()
    init = tf.global_variables_initializer()
    print('init netwrok from scratch....')
    sess.run(init)
    

    # restore experts or build train saver
    expert1_varlist = {v.op.name[8:]: v
          for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="expert1/")}
    expert1_saver = tf.train.Saver(var_list=expert1_varlist)

    expert2_varlist = {v.op.name[8:]: v
          for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="expert2/")}
    expert2_saver = tf.train.Saver(var_list=expert2_varlist)

    expert3_varlist = {v.op.name[8:]: v
          for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="expert3/")}
    expert3_saver = tf.train.Saver(var_list=expert3_varlist)

    # restore the saver
    expert1_saver.restore(sess, model1)

    expert2_saver.restore(sess, model2)
    
    expert3_saver.restore(sess, model3)
    
    # build and restore gating network 
    gating_varlist = {v.name.lstrip("gating_network/"): v
    for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="gating_network/")}
    gating_saver = tf.train.Saver(var_list=gating_varlist, max_to_keep=100)
    
    if FLAGS.restore_gating:
        gating_saver.restore(sess, model5)
        
    # restore back to pixel (H)
    back2pixcel_varlist = {'pred_rnn/'+ v.op.name[15:]: v
      for v in tf.get_collection(tf.GraphKeys.VARIABLES, scope="gating_network/back_to_pixel")}
    print(back2pixcel_varlist)
    back2pixel_saver = tf.train.Saver(var_list=back2pixcel_varlist)
    back2pixel_saver.restore(sess, model4)
    
    # build all saver
    all_varlist = {v.name: v
    for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)}
    all_saver =  tf.train.Saver(all_varlist, max_to_keep=100)

    if FLAGS.restore_all:
        all_saver.restore(sess, model6)
            
    Loss, SSIM_test = [], []
    Pred, w_G_l = [], []
    
    # test
    for idx in range(0, test_data.shape[0], FLAGS.batch_size):
        test_da = test_data[idx:idx+FLAGS.batch_size, :, :, :].transpose(0, 2, 3, 1)
        MSE_loss, x_gene, w_gene_l, SSIM_loss =  sess.run([MSE, x_g, weights_last, SSIM ], feed_dict = {x:test_da})
        Loss.append(MSE_loss)
        Pred.append(x_gene)
        w_G_l.append(w_gene_l)
        SSIM_test.append(SSIM_loss)
        
    return Loss, Pred, w_G_l, SSIM_test

In [ ]:
def main():
    model1 = './models/STMoE-2_trans'
    model2 = './models/STMoE-2_rot'
    model3 = './models/STMoE-2_grow' 
    model4 = './models/STMoE-2_H' 
    model5 = './models/STMoE-2_gating'
    model6 = './models/STMoE-2_all'

    test_data = np.load('./data/test_data.npy')
    max_intensity = 255
        
    test_data = test_data / max_intensity
    
    Loss, Pred, w_G_l, SSIM_test = test(model1, model2, model3, model4, model5, model6, test_data)
    
    return Loss, Pred, w_G_l, SSIM_test, test_data

In [ ]:
if __name__ == '__main__':
    Loss, Pred, w_G_l, SSIM_test, test_data = main()

In [ ]:
%matplotlib inline

## Loss

In [ ]:
Loss = np.array(Loss)*255**2
Loss.mean()

In [ ]:
SSIM = np.array(SSIM_test)
SSIM.mean()

## Visualization

In [ ]:
Pred = np.array(Pred)*255
Pred = Pred[:, 0, :, :, :]
test_data = test_data*255

In [ ]:
num = 79
vmx = 255

In [ ]:
## pred vs gt

In [ ]:
for i in range(FLAGS.seq_length):   
    if i >= FLAGS.seq_start:
        plt.subplot(1, 2, 2)
        plt.imshow(Pred[num, :, :, i-FLAGS.seq_start], vmin = 0, vmax = vmx, cmap = 'jet') 
        plt.title('est_{}'.format(i))
        plt.colorbar()
    plt.subplot(1, 2, 1)
    plt.imshow(test_data[num, i, :, :], vmin = 0, vmax = vmx, cmap = 'jet')   
    plt.title('gt_{}'.format(i))
    plt.colorbar()
    plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(FLAGS.seq_length):   
    if i >= FLAGS.seq_start:
        plt.subplot(1, 10, i+1)
        plt.imshow(Pred[num, :, :, i-FLAGS.seq_start], vmin = 0, vmax = vmx, cmap = 'jet') 
        plt.title('est_{}'.format(i))
plt.show()

## pred and contribution of each experts

In [ ]:
w_g_l = np.array(w_G_l)
w_g_l = w_g_l[:, :, 0]
w_g_l.shape
w_g_l_ = abs(w_g_l) / np.sum(abs(w_g_l), axis = 4)[:, :, :, :, np.newaxis]

In [ ]:
nums = [79]
k = ['translation', 'rotation', 'g/d']
t = 0
axes = []
fig=plt.figure(figsize=(12, 9))
for i, n in enumerate(nums):
    for j in range(3):
        axes.append(fig.add_subplot(4, 4, ((4*i+j+1))))
        plt.imshow(Pred[n, :, :, t], cmap ='jet', vmin=0, vmax=vmx)
        plt.imshow(abs(w_g_l_[n, t, :, :, j])/(w_g_l_.max()), alpha=0.6, vmax= 0.8)
        plt.title('{}'.format(k[j]), fontsize=15)
    plt.colorbar()